In [1]:
import pandas as pd
import numpy as np
import random
from itertools import combinations, permutations
from numpy import dot
from numpy.linalg import norm

In [2]:
df = pd.read_csv('../../data/model/final!!!!!!!!!!!!!!!!!!!!!!!.csv', index_col=0)
dist_mat = pd.read_csv('../../data/model/distance_matrix_final.csv', index_col=0)
df_food_visit = pd.read_csv('../../data/model/food_with_co2_final.csv', index_col=0)
df_acco_visit = pd.read_csv('../../data/model/stay_tagged.csv')

In [3]:
df.head()

,이름,purpose1,purpose2,purpose3,popularity,co2,loc1,loc2,주변관광지,주변음식점,주변숙박
0,성산일출봉(UNESCO 세계자연유산),0.771565,0.682662,0.948563,0.692564,7.606409,15,13,"['성산일출봉(UNESCO 세계자연유산)', '광치기해변', '우도잠수함(제주씨월드...","['맛나식당', '성산회관', '경미네집(경미휴게소)', '바다의집', '그리운바다...","['아름다운리조트', '플레이스 캠프 제주', '골든튤립 제주 성산호텔', '봄그리..."
1,사려니숲길,0.376859,0.325865,0.345421,0.693147,4.831689,1,0,"['사려니숲길', '제주돌문화공원', '노루생태관찰원', '샤이니숲길', '물찻오름...",[],[]
2,카멜리아힐,0.374966,0.568843,0.391210,0.689060,3.687691,8,4,"['카멜리아힐', '아라고나이트 온천(디아넥스호텔)', '방주교회', '헬로키티아일...","['아뜰리에 제주명월', '숨비나리카페', '창천삼거리식당', '숨비나리', '헬로...","['디아넥스 호텔', '중문훼미리리조트', '서귀포호텔 카라반', '레이크힐스 제주..."
3,협재해수욕장,0.740964,0.488754,0.494228,0.689645,1.639161,5,4,"['협재해수욕장', '한림공원', '금능해수욕장', '비양도', '월령선인장군락지'...","['명랑스낵', '피어22', '협재해녀의집', '앤트러사이트', '수우동', '명...","['inn jeju 인제주 게스트하우스', '객의하우스', '블루하와이리조트', '..."
4,월정리해변,0.625939,0.477449,0.401614,0.687889,1.639161,16,13,"['만장굴', '월정리해변', '월정리 카약체험장', '해맞이해안로', '타보카 월...","['구좌상회', '슬슬슬로우', '민경이네어등포해녀촌', '카페콧수염', '월정리에...","['소풍(월정리)', '동박생이', '여울목게스트하우스(월정리)', '뒹글하우스',..."


In [4]:
dist_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,848,849,850,851,852,853,854,855,856,857
0,0.000000,27.982931,56.188593,65.252372,16.427955,26.460400,23.224645,54.900581,62.633203,41.773831,...,44.580294,26.436862,3.991053,64.364949,64.505670,65.275245,62.505821,35.363124,81.681544,47.229609
1,27.982931,0.000000,28.656421,37.526944,21.859265,15.275406,5.210426,26.936847,34.744130,13.835738,...,19.821806,10.440169,27.020414,37.323532,36.535963,37.520627,34.974590,10.388595,69.675450,20.271257
2,56.188593,28.656421,0.000000,16.616512,49.650862,39.639718,33.831069,8.625210,7.564904,16.370206,...,24.555824,31.314689,54.603683,9.164306,10.431407,16.329368,16.712549,21.346259,74.915033,17.946397
3,65.252372,37.526944,16.616512,0.000000,54.974608,43.231046,42.034544,11.326191,10.777774,23.762731,...,23.441149,43.634991,64.535818,19.348015,8.195763,0.392893,3.397650,33.925973,63.237358,18.466577
4,16.427955,21.859265,49.650862,54.974608,0.000000,12.134808,16.854528,46.038325,54.825207,33.496413,...,32.213764,26.780620,18.916690,58.631971,56.069317,55.080568,51.861401,31.977439,65.304982,36.518957


In [6]:
df_food_visit.head()

,이름,지역,태그,주소,좋아요,찜하기,리뷰,방문했어요,조회,SNS공유,...,오메기,라면,카페,술,기타,cat_num,cluster_num,co2,pop,co2_rank
0,봄날카페,제주시 > 애월,#카페 #커피 #음료\n#음식 #콜드브루 #아메리카노 #바닐라라떼 #카페라떼 #아인...,제주특별자치도 제주시 애월읍 애월로1길 25,46,644,141,2,"63,495",62,...,0,0,1,0,0,1,1,0.350000,0.692525,0.614967
1,서연의집(영화 건축학개론),서귀포시 > 남원,#카페 #커피 #아메리카노\n#음식 #디저트 #빵 #와플 #에스프레소 #카페모카 #...,제주특별자치도 서귀포시 남원읍 위미해안로 86,14,252,68,2,"40,283",21,...,0,0,1,0,0,1,1,0.350000,0.691281,0.614967
2,명진전복,제주시 > 구좌,#전복 #돌솥밥 #회\n#음식 #전복해물탕 #전복죽 #전복뚝배기 #전복버터구이 #전복물회,제주특별자치도 제주시 구좌읍 해맞이해안로 1282,32,640,66,0,"49,310",33,...,0,0,0,0,0,4,4,2.230000,0.692214,0.165715
3,몽상드애월(몽상),제주시 > 애월,#카페 #커피 #음료\n#디저트 #음식 #아메리카노 #에스프레소 #카페모카 #카라멜...,제주특별자치도 제주시 애월읍 애월북서길 56-1,21,492,66,1,"48,176",33,...,0,0,1,0,0,1,1,0.350000,0.691903,0.614967
4,제주김만복(만복이네김밥집),제주시 > 제주시내,#전복김밥 #통전복주먹밥 #미역국\n#음식 #해물라면 #라면,제주특별자치도 제주시 오라로 41,75,"1,182",60,1,"77,618",123,...,0,1,0,0,1,3,3,2.196667,0.693147,0.174888


In [7]:
df_acco_visit.head()

,이름,지역,태그,주소,좋아요,찜하기,리뷰,방문했어요,조회,SNS공유,...,상세정보,등급,편의시설,기타상세,상세텍스트,주변관광지개수,주변관광지,주변음식점개수,주변음식점,숙박업종
0,나미송 머무는 곳,제주시 > 애월,#민박 #고향민박 #독채\n#정원 #주방기구 #자연경관,제주특별자치도 제주시 애월읍 평화로 2476,25,27,159,59,"34,811",74,...,NaN,NaN,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,음료대,경보 및 피난시설",NaN,상세정보\n확장됨\n\n애월읍 시인의 마을에 위치한 나미송 머무는 곳 민박은 요즘 ...,4,"['제주불빛정원 테마파크', '제주승마공원', '큰노꼬메오름', '족은노꼬메오름']",2,"['1950하우스', '참솔식당']",펜션/민박
1,해비치호텔앤드리조트,서귀포시 > 표선,#휴식/치유 #5성급호텔 #호텔\n#숙소 #리조트 #가족호텔 #공공와이파이존 #양식...,제주특별자치도 서귀포시 표선면 민속해안로 537,14,75,45,1,"37,922",39,...,NaN,5성급,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,음료대,유도 및 ...",NaN,상세정보\n확장됨\n\n해가 처음 비추는 곳이라는 뜻의 해비치 호텔&리조트는 에메랄...,12,"['제주민속촌', '제주허브동산', '표선해수욕장', '아이리스촬영지(하얀등대)',...",24,"['돌토고리족욕카페', '성신흑돼지', '골목집갈치와고등어', '탐라 뚝배기', '...",호텔
2,신라호텔제주,서귀포시 > 중문,#최고급 #VIP #중문\n#신라 #중문관광단지 #5성급호텔 #숙소 #게스트하우스 ...,제주도 서귀포시 중문관광로 72번길 75,7,64,43,0,"25,150",12,...,NaN,NaN,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,음료대,유도 및 ...",NaN,상세정보\n확장됨\n\n중문관광단지에 위치한 제주 신라호텔은 90년 개관 이후 이국...,33,"['주상절리대(중문대포해안)', '천제연폭포', '제주실탄사격장', '중문색달해수욕...",36,"['고집돌우럭(중문)', '바다다', '수두리보말칼국수', '흑돼지구이집 하영', ...",게스트하우스
3,롯데호텔 제주,서귀포시 > 중문,#호텔 #5성급 #중문\n#리조트 #숙소 #가족호텔 #수영장 #양식레스토랑 #자연경...,제주특별자치도 서귀포시 중문관광로72번길 35,9,47,38,1,"24,762",19,...,NaN,5성급,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,음료대,유도 및 ...",NaN,상세정보\n확장됨\n\n푸른 제주바다와 하늘이 맞닿은 곳에 위치한 롯데호텔 제주는 ...,34,"['주상절리대(중문대포해안)', '천제연폭포', '제주실탄사격장', '중문색달해수욕...",38,"['고집돌우럭(중문)', '바다다', '수두리보말칼국수', '흑돼지구이집 하영', ...",호텔
4,제주조천스위스마을게스트하우스,제주시 > 조천,#숙박 #숙소 #게스트하우스\n#마을관광 #올레길 #식당,제주특별자치도 제주시 조천읍 함와로 566-27,12,49,29,0,"51,343",50,...,NaN,NaN,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,유도 및 안내시설...",NaN,"상세정보\n확장됨\n\n조천읍 와산리에 위치한 제주조천스위스마을은 총 4개 단지, ...",17,"['제주조천스위스마을', '선녀와나무꾼', '동굴의 다원 다희연', '제주라프 짚라...",11,"['낭뜰에쉼팡', '카페동백', '샤라의정원', '고사리식당', '선흘방주할매식당'...",게스트하우스
